# Transfer learning

![transfer learing](https://cdn-images-1.medium.com/max/1600/1*yTJ6h-kJVJ1goDEvNd07xg.png)

## Essential import

In [0]:
from __future__ import print_function

import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

## Initialize some parameters

In [0]:
now = datetime.datetime.now

batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

## Check channel_first

In [0]:
if K.image_data_format() == 'channels_first':
    input_shape = (1, img_rows, img_cols)
else:
    input_shape = (img_rows, img_cols, 1)

## Train function

In [0]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    t = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    print('Training time: %s' % (now() - t))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

## Load dataset

In [0]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

## Network arch

In [0]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    Conv2D(filters, kernel_size,
           padding='valid',
           input_shape=input_shape),
    Activation('relu'),
    Conv2D(filters, kernel_size),
    Activation('relu'),
    MaxPooling2D(pool_size=pool_size),
    Dropout(0.25),
    Flatten(),
]

classification_layers = [
    Dense(128),
    Activation('relu'),
    Dropout(0.5),
    Dense(num_classes),
    Activation('softmax')
]

# create complete model
model = Sequential(feature_layers + classification_layers)

## Train Model

In [16]:
# train model for 5-digit classification [0..4]
train_model(model,(x_train_lt5, y_train_lt5),(x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 5s 176us/step - loss: 0.1711 - acc: 0.9461 - val_loss: 0.0409 - val_acc: 0.9875
Epoch 2/5
30596/30596 [==============================] - 3s 111us/step - loss: 0.0448 - acc: 0.9865 - val_loss: 0.0147 - val_acc: 0.9949
Epoch 3/5
30596/30596 [==============================] - 3s 111us/step - loss: 0.0296 - acc: 0.9910 - val_loss: 0.0081 - val_acc: 0.9971
Epoch 4/5
30596/30596 [==============================] - 3s 112us/step - loss: 0.0240 - acc: 0.9926 - val_loss: 0.0076 - val_acc: 0.9979
Epoch 5/5
30596/30596 [==============================] - 3s 112us/step - loss: 0.0190 - acc: 0.9944 - val_loss: 0.0061 - val_acc: 0.9979
Training time: 0:00:19.603693
Test score: 0.006117485956124683
Test accuracy: 0.9978595057404164


## Freeze feature layers and rebuild model

In [0]:
for l in feature_layers:
  l.trainable = False

## Transfer: train dense layers for new classification task [5..9]

In [18]:
train_model(model,(x_train_gte5, y_train_gte5),(x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 3s 91us/step - loss: 0.2318 - acc: 0.9331 - val_loss: 0.0479 - val_acc: 0.9835
Epoch 2/5
29404/29404 [==============================] - 2s 76us/step - loss: 0.0721 - acc: 0.9771 - val_loss: 0.0330 - val_acc: 0.9891
Epoch 3/5
29404/29404 [==============================] - 2s 75us/step - loss: 0.0557 - acc: 0.9830 - val_loss: 0.0284 - val_acc: 0.9895
Epoch 4/5
29404/29404 [==============================] - 2s 75us/step - loss: 0.0451 - acc: 0.9860 - val_loss: 0.0288 - val_acc: 0.9903
Epoch 5/5
29404/29404 [==============================] - 2s 75us/step - loss: 0.0420 - acc: 0.9878 - val_loss: 0.0240 - val_acc: 0.9922
Training time: 0:00:11.870938
Test score: 0.023993675379197322
Test accuracy: 0.992182678461222


# Exercise for you

Design a image classifier for cifar10 dataset with transfer learning. 

## Tasks:
* Train the network on any 5 categories 
* Display the train and test accuracy
* Use transfer learning to train the rest of the 5 categories oon the same network
* Display the train and test accuracy after tranfer learning

Use the same above code and modify to this problem.

In [0]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()